In [31]:
#Package installations for GoogleColab
!pip install deeplake
!pip install "deeplake[enterprise]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 14.9 MB/s eta 0:00:00


In [6]:
import deeplake

In [35]:
ds = deeplake.load('hub://activeloop/fgnet')

train_ds, val_ds = ds.random_split([0.8, 0.2])

print(len(train_ds), len(val_ds))

hub://activeloop/fgnet loaded successfully.

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/fgnet

802 200


In [8]:
dataloader = ds.pytorch(num_workers=0, batch_size=4, shuffle=False)

/usr/local/lib/python3.9/dist-packages/deeplake/integrations/pytorch/common.py:91: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


In [9]:
dataloader = ds.tensorflow()

In [33]:
ds.summary()

Dataset(path='hub://activeloop/fgnet', read_only=True, index=Index([(537, 436, 700, 419, 430, 579, 403, 441, 656, 501, 654, 641, 507, 682, 681, 469, 607, 474, 449, 364, 371, 446, 363, 443, 669, 678, 554, 632, 699, 406, 381, 691, 506, 668, 659, 680, 369, 374, 566, 602, 461, 676, 409, 576, 571, 496, 564, 580, 476, 502, 696, 558, 585, 646, 689, 606, 636, 373, 516, 693, 530, 553, 447, 610, 519, 698, 413, 407, 418, 378, 639, 683, 465, 529, 589, 493, 635, 423, 638, 542, 489, 459, 415, 660, 440, 708, 565, 695, 379, 548, 428, 396, 479, 390, 706, 675, 361, 556, 387, 626, 697, 368, 426, 591, 540, 679, 427, 622, 498, 359, 372, 527, 462, 421, 686, 583, 655, 429, 532, 640, 650, 645, 597, 561, 703, 599, 424, 360, 662, 500, 664, 702, 557, 535, 624, 652, 401, 408, 486, 690, 653, 405, 594, 435, 467, 365, 458, 484, 608, 412, 620, 512, 631, 649, 454, 538, 410, 411, 567, 544, 550, 466, 475, 671, 648, 582, 560, 673, 634, 704, 590, 677, 367, 522, 524, 573, 630, 457, 455, 586, 402, 572, 685, 545, 400, 555, 6

In [11]:
ds.visualize()

HINT: Please forward the port - 34963 to your local machine, if you are running on the cloud.
 * Serving Flask app 'dataset_visualizer'


 * Debug mode: off


In [19]:
#Strictly observational

image = ds.images[0].numpy() # Fetch the first image and return a numpy array
labels = ds.ages[0].data() # Fetch the labels in the first image

# Slicing
img_list = ds.ages[0:100].numpy(aslist=True) # Fetch 100 labels and store 
                                               # them as a list of numpy arrays



Building A sample Neural Net

In [37]:
# converting training images into torch format
train_x = train_x.reshape(54000, 1, 28, 28)
train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_y.astype(int);
train_y = torch.from_numpy(train_y)

# shape of training data
train_x.shape, train_y.shape

In [41]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

In [60]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.ToPILImage(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainloader= train_ds.pytorch(num_workers=0, batch_size=4, shuffle=False, transform=transform)

testloader= val_ds.pytorch(num_workers=0, batch_size=4, shuffle=False, transform=transform)

/usr/local/lib/python3.9/dist-packages/deeplake/integrations/pytorch/common.py:91: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


In [43]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [44]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [61]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

TransformFailedError: ignored